- 다중 질의 생성 과정을 모니터링 하기 위한 로깅을 설정합니다.

In [1]:
# 쿼리를 위한 로그 설정
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

- 벡터 데이터베이스와 문서로더를 설정한뒤 문서를 불러옵니다.

In [4]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문서 로더 설정
loaders = [TextLoader("data/How_to_invest_money.txt", encoding="utf-8")]

docs = []
for loader in loaders:
    docs.extend(loader.load())

- 텍스트 분할기와 벡터 저장소 인스턴스를 생성합니다.
- 텍스트를 분할합니다.

In [5]:
# 문서 생성을 위한 텍스트 분할기 정의
recursive_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# 문서 분할
split_docs = recursive_splitter.split_documents(docs)

embeddings = OllamaEmbeddings(model="bge-m3")

# chroma vectorstore 생성
vectorstore = Chroma.from_documents(
    documents=split_docs,
    embedding=embeddings,
)

- 다중 질의 생성을 위한 리트리버를 설정합니다.

In [6]:
from langchain.retrievers import MultiQueryRetriever
from langchain_openai import ChatOpenAI

# LLM 모델 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

# MultiQueryRetriever 실행
retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), # 기본 검색기
    llm=llm,
)

In [7]:
# 예시 질문
question = "주식 투자를 처음 시작하려면 어떻게 해야 하나요?"

# 결과 검색
unique_docs = retriever.invoke(question)
print(f"\n 결과: {len(unique_docs)}개의 문서가 검색되었습니다.")

INFO:langchain.retrievers.multi_query:Generated queries: ['주식 투자를 처음 시작할 때 어떤 단계가 필요한가요?  ', '주식 투자에 입문하기 위해 알아야 할 기본 사항은 무엇인가요?  ', '초보자가 주식 투자를 시작하기 위해 준비해야 할 것은 무엇인가요?']



 결과: 7개의 문서가 검색되었습니다.


- 검색된 문서들을 활용하여 최종 답변을 생성하는 RetrievalQA 체인을 설정하겠습니다.

In [9]:
from langchain.chains.retrieval_qa.base import RetrievalQA

# RetrievalQA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

# 질문에 대한 답변 생성
result = qa_chain.invoke({"query": question})

# 결과 출력
print("답변:", result["result"])
print("\n사용된 문서:")
for doc in result["source_documents"]:
    print(doc.page_content)   

INFO:langchain.retrievers.multi_query:Generated queries: ['주식 투자에 처음 입문할 때 어떤 단계부터 시작해야 할까요?  ', '주식 투자를 시작하기 위해 필요한 기본 지식이나 팁은 무엇인가요?  ', '초보자가 주식 투자에 성공하기 위해 알아야 할 중요한 사항은 무엇인가요?']


답변: 주식 투자를 처음 시작하려면 다음과 같은 단계를 고려할 수 있습니다:

1. **기본 이해**: 주식과 채권의 차이, 투자 원칙, 시장의 기본 작동 방식 등을 이해합니다.

2. **목표 설정**: 투자 목표를 명확히 하고, 자신의 재정적 요구와 상황을 평가합니다. 어떤 목적을 가지고 투자하는지에 따라 전략이 달라질 수 있습니다.

3. **교육**: 주식 투자에 대한 책이나 온라인 강좌를 통해 더 많은 정보를 습득합니다. 주식 시장의 기본 원리와 투자 전략에 대한 지식을 쌓는 것이 중요합니다.

4. **시장 조사**: 투자할 주식이나 산업에 대한 조사를 진행합니다. 기업의 재무 상태, 시장 동향, 경쟁사 분석 등을 포함합니다.

5. **투자 계좌 개설**: 주식 거래를 위한 증권 계좌를 개설합니다. 여러 증권사에서 제공하는 다양한 계좌 옵션을 비교해 보세요.

6. **소액 투자 시작**: 처음에는 소액으로 시작하여 경험을 쌓고, 시장의 움직임을 관찰합니다.

7. **리스크 관리**: 투자할 때는 리스크를 분산시키고, 손실을 감당할 수 있는 범위 내에서 투자합니다.

8. **지속적인 학습**: 시장의 변화에 따라 지속적으로 학습하고, 투자 전략을 조정합니다.

이러한 단계를 통해 주식 투자에 대한 기초를 다지고, 성공적인 투자자가 될 수 있는 기반을 마련할 수 있습니다.

사용된 문서:
The foregoing comments, in a brief and imperfect way, serve to indicate
the main points which should be considered in the selection of
securities for investment. The considerations advanced will be amplified
as occasion demands in the following pages. For the present, the main
lesson which it is sought to draw is th